### Importing Libraries

In [2]:
import numpy
import pandas
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import scale

### Preprocessing

In [3]:
# Adding channels
def summed_data(data):
    return data[0] + data[1] + data[2] + data[3] + data[4] + data[5] + data[6] + data[7] + data[8] + data[9]

def butter_LowPass(series, N, Wn):
    # N = 2  # Filter order
    # Wn = 0.01  # Cutoff frequency
    B, A = butter(N, Wn, output='ba')
    y = filtfilt(B, A, series)
    return y

In [4]:
def create_epochs(t_series, epoch_length):
    list_a = []
    for j in range(0, len(t_series), epoch_length):
        epoch = t_series[j : j + epoch_length]
        list_a.append(epoch)
    return list_a

# creating epoch of all actions 
def create_epoch_partedEvents(epo_series):
    events = []
    for j in range(len(epo_series)):
        rest = epo_series[j][0:5000]
        stimulus = epo_series[j][5000:6000]
        get_ready = epo_series[j][6000:8000]
        imagined_speech = epo_series[j][8000:13000]
        speak = epo_series[j][13000:14000]
        events.append(rest)
        events.append(stimulus)
        events.append(get_ready)
        events.append(imagined_speech)
        events.append(speak)
    return events

In [5]:
def time_series():
    person1 = create_epoch_partedEvents(
        create_epochs(butter_LowPass(summed_data(numpy.load("Data/Person1.npy")), 2, 0.01), 14000))
    person2 = create_epoch_partedEvents(
        create_epochs(butter_LowPass(summed_data(numpy.load("Data/Person2.npy")), 2, 0.01), 14000))
    person4 = create_epoch_partedEvents(
        create_epochs(butter_LowPass(summed_data(numpy.load("Data/Person4.npy")), 2, 0.01), 14000))
    return person1, person2, person4

In [6]:
a, b, d = time_series()

In [10]:
# Extracting imagined speech part
def final_data(series):
    val = []
    indexes = numpy.arange(3, len(series), 5)
    for i in indexes:
        val.append(series[i])
    val = val[0:145]
    return numpy.array(val)

In [11]:
a1 = final_data(a)
b1 = final_data(b)
d1 = final_data(d)

In [12]:
y = numpy.vstack((a1, b1, d1))

In [13]:
def class_seperator(series, start, end, step):
    b = []
    a = numpy.arange(start, end, step)
    for i in a:
        b.append(series[i])
    return numpy.array(b)

In [14]:
# Separating the classes 
iy = class_seperator(y, 0, len(y), 11)
print(iy.shape)

uw = class_seperator(y, 1, len(y), 11)
print(uw.shape)

piy = class_seperator(y, 2, len(y), 11)
print(piy.shape)

tiy = class_seperator(y, 3, len(y), 11)
print(tiy.shape)

diy = class_seperator(y, 4, len(y), 11)
print(diy.shape)

m = class_seperator(y, 5, len(y), 11)
print(m.shape)

n = class_seperator(y, 6, len(y), 11)
print(n.shape)

pat = class_seperator(y, 7, len(y), 11)
print(pat.shape)

pot = class_seperator(y, 8, len(y), 11)
print(pot.shape)

knew = class_seperator(y, 9, len(y), 11)
print(knew.shape)

gnaw = class_seperator(y, 10, len(y), 11)
print(gnaw.shape)

fg = numpy.vstack((iy, uw, piy, tiy, diy, m, n, pat, pot, knew, gnaw))
fg = scale(fg)

print(fg.shape)

(40, 5000)
(40, 5000)
(40, 5000)
(40, 5000)
(40, 5000)
(40, 5000)
(39, 5000)
(39, 5000)
(39, 5000)
(39, 5000)
(39, 5000)
(435, 5000)


In [15]:
# creating data frame 
df = pandas.DataFrame(fg)
df = df.sample(frac=1)
df.to_csv("final_data.csv")